# Motion Vector Generator datapath generator

#### Parameters

In [1]:
import math 
import numpy as np
from datetime import datetime

In [5]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS
BUFFER_LINES = 9                                # Number of lines in the transposed buffer
BUFFER_COLUMNS = 4                              # Number of columns in the transposed buffer


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)


#### Intermediate variables calculation

In [28]:
####################################################
# MV GENERATOR DATAPATH DEFINITIONS
####################################################
MV_GEN_FILE_NAME = 'MV_gen_datapath'            # Interpolation datapath file name
DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

# Control inputs [signal name, bit width, io type, description]
# Obs: io type is relative to this datapath pov
MV_GEN_CTRL_SIGNALS = [
    ['RST_ASYNC_MV_GEN', 1, 'input', '\t\t\t// Set of reg resets: coord_X, coord_Y, X, Y, MV_0, MV_1, gen_MV_X, gen_MV_Y and count_block.'], 
    # ['rst_async_reg_count_block', 1, 'input', ],
    ['WRITE_REGS_COORDS', 1, 'input', '\t\t\t// Enable writing to registers coord_X and coord_Y.'], 
    ['WRITE_REGS_MVS', 1, 'input', '\t\t\t\t// Enable writing to registers MV_0 and MV_1.'], 
    ['WRITE_REGS_GEN_MVS', 1, 'input', '\t\t\t// Enable writing to registers gen_MV_X and gen_MV_Y.'], 
    ['WRITE_REG_X', 1, 'input', '\t\t\t\t// Enable writing for reg_X.'], 
    ['WRITE_REG_Y', 1, 'input', '\t\t\t\t// Enable writing for reg_Y.'], 
    ['WRITE_REG_COUNT_BLOCK', 1, 'input', '\t\t\t// Enable writing for reg_count_block.'], 
    ['SEL_X', 1, 'input', '\t\t\t\t\t// Select: 0 - original X coord; 1: incremented X coord.'], 
    ['SEL_Y', 1, 'input', '\t\t\t\t\t// Select: 0 - original Y coord; 1: incremented Y coord.'], 
    ['COORD_X', INPUT_COORD_WIDTH, 'input', '\t\t\t\t// First X coordenate.'],
    ['COORD_Y', INPUT_COORD_WIDTH, 'input', '\t\t\t\t// First Y coordenate.'],
    ['MV_0', 2*INPUT_COORD_WIDTH, 'input', '\t\t\t\t// Motion vector 0. MSB = x component, LSB = y component.'],
    ['MV_1', 2*INPUT_COORD_WIDTH, 'input', '\t\t\t\t// Motion vector 1. MSB = x component, LSB = y component.'],  
    ['INTERP_X', 1, 'output wire', '\t\t\t\t// Interpolate horizontally? 0: No. 1: Yes.'], 
    ['INTERP_Y', 1, 'output wire', '\t\t\t\t// Interpolate vertically? 0: No. 1: Yes.'], 
    ['CTRL_X', 1, 'output wire', '\t\t\t\t// {ctrl_X, ctrl_Y}? 00: Done all. 01: Next column. 10: Next line. 11: Next column.'], 
    ['CTRL_Y', 1, 'output wire', '\t\t\t\t// {ctrl_X, ctrl_Y}? 00: Done all. 01: Next column. 10: Next line. 11: Next column.']
]

# Inter

# Components informations [component name, component type, [inputs names], [inputs widths], [outputs], [outputs width]]
# ['', '', [], [], [], []],
MV_GEN_COMPONENTS = [
    ['ADDX', 'add_coords', [str(INPUT_COORD_WIDTH)+"'b1", ], [INPUT_COORD_WIDTH, INPUT_COORD_WIDTH], [], [INPUT_COORD_WIDTH]],
    []
]

In [29]:
####################################################
# MV GENERATOR DATAPATH CALCULATIONS
####################################################

# Inputs declaration
IO_DECLARATION = []
for i in range(len(MV_GEN_CTRL_SIGNALS)):
    if MV_GEN_CTRL_SIGNALS[i][1] == 1:
        if i != len(MV_GEN_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' '+MV_GEN_CTRL_SIGNALS[i][0]+','+MV_GEN_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' '+MV_GEN_CTRL_SIGNALS[i][0]+MV_GEN_CTRL_SIGNALS[i][3])
    else:
        if i != len(MV_GEN_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' ['+str(MV_GEN_CTRL_SIGNALS[i][1]-1)+':0] '+MV_GEN_CTRL_SIGNALS[i][0]+','+MV_GEN_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' ['+str(MV_GEN_CTRL_SIGNALS[i][1]-1)+':0] '+MV_GEN_CTRL_SIGNALS[i][0]+MV_GEN_CTRL_SIGNALS[i][3])    
IO_DECLARATION_CONCAT = '\n'.join(IO_DECLARATION)

# print(IO_DECLARATION_CONCAT)

#### Datapath text 

In [26]:
#########################################
# Text writing: MV Gen datapath
#########################################

MV_GEN_DATAPATH_VERILOG = '''/*-----------------------------------------------------------------------------------
* File: '''+MV_GEN_FILE_NAME+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: Motion Vector Generator datapath.
*----------------------------------------------------------------------------------- */

module '''+MV_GEN_FILE_NAME+''' (
'''+IO_DECLARATION_CONCAT+'''
);


// ------------------------------------------
// Wires declarations
// ------------------------------------------


// ------------------------------------------
// Modules instantiation
// ------------------------------------------


// ------------------------------------------
// Combinational logic
// ------------------------------------------


// ------------------------------------------
// Outputs
// ------------------------------------------



endmodule // '''+MV_GEN_FILE_NAME+'''
'''

print(MV_GEN_DATAPATH_VERILOG)

/*-----------------------------------------------------------------------------------
* File: MV_gen_datapath.v
* Date generated: 25/03/2023
* Date modified: 10/05/2023
* Author: Bruna Suemi Nagai
* Description: Motion Vector Generator datapath.
*----------------------------------------------------------------------------------- */

module MV_gen_datapath (
  input RST_ASYNC_MV_GEN,			// Set of reg resets: coord_X, coord_Y, X, Y, MV_0, MV_1, gen_MV_X, gen_MV_Y and count_block.
  input WRITE_REGS_COORDS,			// Enable writing to registers coord_X and coord_Y.
  input WRITE_REGS_MVS,				// Enable writing to registers MV_0 and MV_1.
  input WRITE_REGS_GEN_MVS,			// Enable writing to registers gen_MV_X and gen_MV_Y.
  input WRITE_REG_X,				// Enable writing for reg_X.
  input WRITE_REG_Y,				// Enable writing for reg_Y.
  input WRITE_REG_COUNT_BLOCK,			// Enable writing for reg_count_block.
  input SEL_X,					// Select: 0 - original X coord; 1: incremented X coord.
  input SEL_Y,					// Sele